# Notebook 01: Ingesta y Normalización de Datos Energéticos

**Objetivo:** Transformar los datos crudos de consumo eléctrico (CNE) en un dataset limpio, estandarizado y listo para ser cruzado (merge) con otras fuentes.

**Input:** `data/raw/consumo_electrico_cne_2024.xlsx` (Archivo con problemas de formato y codificación).

**Output:** `data/processed/consumo_electrico_limpio.csv` (Datos limpios, filtrados por RM y normalizados).

**Pasos Clave:**
1. **Parseo:** Separar columnas unidas por punto y coma.
2. **Corrección de Texto:** Arreglar errores de codificación (ej: `RegiÃ³n` -> `Región`).
3. **Tipado:** Forzar números y fechas a sus tipos correctos.
4. **Normalización:** Estandarizar nombres de comunas (minúsculas, sin tildes) para garantizar cruces exitosos.
5. **Filtrado:** Seleccionar solo la Región Metropolitana.

In [1]:
import pandas as pd
import numpy as np
import os
import unicodedata

# Configuración de Rutas
INPUT_PATH = os.path.join('..', 'data', 'raw', 'consumo_electrico_cne_2024.xlsx')
OUTPUT_PATH = os.path.join('..', 'data', 'processed', 'consumo_electrico_limpio.csv')

# Garantizar que el directorio de salida exista
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

## 1. Funciones Auxiliares
Definimos funciones para reparar la codificación (mojibake) y para normalizar los nombres de las comunas.

In [2]:
def arreglar_codificacion(texto):
    """
    Repara errores de codificación comunes (Mojibake) donde UTF-8 se leyó como CP1252.
    Ejemplo: 'RegiÃ³n' -> 'Región', 'MaipÃº' -> 'Maipú'
    """
    if pd.isna(texto):
        return ""
    texto = str(texto)
    try:
        # Codifica a latin-1/cp1252 y decodifica correctamente a utf-8
        return texto.encode('cp1252').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        # Si falla, devolvemos el texto original
        return texto

def normalizar_texto(texto):
    """
    Estandariza strings para facilitar cruces de datos (Llaves primarias):
    1. Convierte a minúsculas.
    2. Elimina espacios al inicio y final.
    3. Elimina tildes (á -> a, ñ -> n).
    """
    if pd.isna(texto):
        return ""
    
    # 1. Minúsculas y stripping
    texto = str(texto).lower().strip()
    
    # 2. Eliminación de tildes usando normalización Unicode (NFKD)
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    
    return texto

## 2. Carga, Parseo y Reparación
Cargamos el Excel, separamos las columnas y aplicamos la corrección de codificación inmediatamente.

In [3]:
print(f"Leyendo archivo desde: {INPUT_PATH}")

try:
    # Cargar el Excel crudo
    df_raw = pd.read_excel(INPUT_PATH)
    
    # Identificar la columna colapsada (la primera)
    col_sucia = df_raw.columns[0]
    print(f"Estructura detectada en columna: {col_sucia[:50]}...")
    
    # Separar la columna única en múltiples columnas
    df = df_raw[col_sucia].str.split(';', expand=True)
    
    # Asignar nombres de columnas limpios
    nombres_cols = col_sucia.split(';')
    df.columns = [c.strip() for c in nombres_cols]

    # --- CORRECCIÓN DE CODIFICACIÓN (NUEVO PASO) ---
    print("Reparando caracteres mal codificados (ej: 'RegiÃ³n')...")
    cols_texto = ['region', 'comuna', 'tipo_clientes', 'tarifa']
    for col in cols_texto:
        if col in df.columns:
            df[col] = df[col].apply(arreglar_codificacion)
    
    print(f"Estructura y texto reparados. Dimensiones: {df.shape}")
    # Mostrar muestra para verificar que 'Región' se lea bien
    display(df[['region', 'comuna']].head(3))

except FileNotFoundError:
    raise FileNotFoundError("❌ No se encontró el archivo raw. Verifica la ruta.")

Leyendo archivo desde: ..\data\raw\consumo_electrico_cne_2024.xlsx
Estructura detectada en columna: anio;mes;region;comuna;tipo_clientes;tarifa;client...
Reparando caracteres mal codificados (ej: 'RegiÃ³n')...
Estructura y texto reparados. Dimensiones: (490758, 10)


,region,comuna
0,Región del Libertador Gral. Bernardo O’Higgins,Las Cabras
1,Región del Libertador Gral. Bernardo O’Higgins,Las Cabras
2,Región del Libertador Gral. Bernardo O’Higgins,Las Cabras


## 3. Limpieza de Tipos de Datos
Convertimos las columnas numéricas.

In [4]:
cols_numericas = ['anio', 'mes', 'clientes_facturados', 'e1_kwh', 'e2_kwh', 'energia_kwh']

print("Convirtiendo columnas numéricas...")
for col in cols_numericas:
    # 'coerce' transforma valores no numéricos en NaN
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

print("Tipos de datos ajustados.")

Convirtiendo columnas numéricas...
Tipos de datos ajustados.


## 4. Normalización y Filtrado Geográfico
Creamos las claves normalizadas (`comuna_norm`) y filtramos por la Región Metropolitana.

In [5]:
print("Normalizando texto para llaves de cruce...")
# Crear columnas normalizadas para el cruce
df['region_norm'] = df['region'].apply(normalizar_texto)
df['comuna_norm'] = df['comuna'].apply(normalizar_texto)

print("Filtrando Región Metropolitana...")
# Usamos 'metropolitana' en la columna normalizada
df_rm = df[df['region_norm'].str.contains('metropolitana', na=False)].copy()

print(f"Filas totales: {len(df)}")
print(f"Filas RM: {len(df_rm)}")

if df_rm.empty:
    raise ValueError("⚠️ ALERTA: El filtrado eliminó todos los datos. Revisa el paso de normalización.")

Normalizando texto para llaves de cruce...
Filtrando Región Metropolitana...
Filas totales: 490758
Filas RM: 74177


## 5. Exportación
Generamos la fecha y guardamos el archivo limpio.

In [6]:
# Crear fecha (primer día del mes) para indexación temporal
df_rm['fecha'] = pd.to_datetime(
    df_rm['anio'].astype(int).astype(str) + '-' + 
    df_rm['mes'].astype(int).astype(str) + '-01'
)

# Selección de columnas finales
cols_finales = [
    'fecha', 'anio', 'mes',
    'comuna_norm',  # Clave limpia para merge
    'comuna',       # Nombre original corregido (para gráficos)
    'region',       # Nombre región corregido
    'tipo_clientes', 'tarifa',
    'clientes_facturados', 'energia_kwh'
]

df_final = df_rm[cols_finales]

# Guardar
print(f"Guardando archivo procesado en: {OUTPUT_PATH}")
df_final.to_csv(OUTPUT_PATH, index=False)

print("✅ Proceso finalizado correctamente.")
display(df_final.head())

Guardando archivo procesado en: ..\data\processed\consumo_electrico_limpio.csv
✅ Proceso finalizado correctamente.


,fecha,anio,mes,comuna_norm,comuna,region,tipo_clientes,tarifa,clientes_facturados,energia_kwh
973,2015-01-01,2015,1,maipu,Maipú,Región Metropolitana de Santiago,No Residencial,BT4.1,1.0,1415
974,2015-01-01,2015,1,maipu,Maipú,Región Metropolitana de Santiago,No Residencial,BT3PPP,55.0,198996
975,2015-01-01,2015,1,maipu,Maipú,Región Metropolitana de Santiago,No Residencial,BT3PP,411.0,1635743
976,2015-01-01,2015,1,maipu,Maipú,Región Metropolitana de Santiago,No Residencial,BT4.2,1.0,1915
977,2015-01-01,2015,1,maria pinto,María Pinto,Región Metropolitana de Santiago,No Residencial,AT2PP,1.0,3127
